### 180107219 - Negmetulla Yerlan

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

import glob

In [4]:
file_path_list = glob.glob('drive/MyDrive/DeepLearningAssignment5/data/*.txt')

full_text = ""
if file_path_list:
    with open(file_path_list[0], 'r') as f:
        full_text += f.read()
    for i in range(len(file_path_list)):
        with open(file_path_list[i], 'r') as f:
            full_text += "\n" + f.read()

In [5]:
chars = tuple(set(full_text))
idx2char = dict(enumerate(chars))
char2idx = {ch: ii for ii, ch in idx2char.items()}

encoded = np.array([char2idx[ch] for ch in full_text])

In [6]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot

In [7]:
def get_batches(arr, batch_size, seq_length):
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size, -1))
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [8]:
class CharRNN(nn.Module):
    def __init__(self, chars, n_hidden=256, n_layers=3, drop_prob=0.5):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.chars = chars
        self.idx2char = dict(enumerate(self.chars))
        self.char2idx = {ch: ii for ii, ch in self.idx2char.items()}
        self.lstm=nn.LSTM(len(self.chars),n_hidden,n_layers, dropout=drop_prob,batch_first=True)
        self.dropout=nn.Dropout(drop_prob)
        self.fc=nn.Linear(n_hidden,len(self.chars))
    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
            weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda()
        )
        return hidden

In [9]:
def train(model, batch, h, criterion, optimizer, clip):
    model.train()
    x, y = batch
    h = tuple([each.data for each in h])
    inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
    inputs, targets = inputs.cuda(), targets.cuda()
    optimizer.zero_grad()
    output, h = model(inputs, h)
    loss = criterion(output, targets.view(batch_size*seq_length).long())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    return loss

In [10]:
def full_train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, print_every=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    n_chars = len(model.chars)
    model.cuda()
    counter = 0
    for e in range(epochs):
        h = model.init_hidden(batch_size)
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            x = one_hot_encode(x, n_chars)
            loss = train(model, (x,y), h, criterion, optimizer, clip)
            if counter % print_every == 0:
                print(f"Epoch: {e+1}/{epochs}, Loss: {loss.item():.4f}")

In [11]:
n_hidden = 512
n_layers = 3
model = CharRNN(chars, n_hidden, n_layers)

In [12]:
model.load_state_dict(torch.load('drive/MyDrive/DeepLearningAssignment5/model.pt'))

<All keys matched successfully>

In [13]:
batch_size = 128
seq_length = 100
n_epochs =  5000
full_train(model, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=500)

Epoch: 500/5000, Loss: 1.0687
Epoch: 1000/5000, Loss: 0.0872
Epoch: 1500/5000, Loss: 0.0411
Epoch: 2000/5000, Loss: 0.0303
Epoch: 2500/5000, Loss: 0.0244
Epoch: 3000/5000, Loss: 0.0246
Epoch: 3500/5000, Loss: 0.0219
Epoch: 4000/5000, Loss: 0.0208
Epoch: 4500/5000, Loss: 0.0201
Epoch: 5000/5000, Loss: 0.0211


In [14]:
torch.save(model.state_dict(), 'drive/MyDrive/DeepLearningAssignment5/model.pt')

In [15]:
def predict(model, char, h=None, top_k=None):
    x = np.array([[model.char2idx[char]]])
    x = one_hot_encode(x, len(model.chars))
    inputs = torch.from_numpy(x)
    inputs = inputs.cuda()
    h = tuple([each.data for each in h])
    out, h = model(inputs, h)
    p = F.softmax(out, dim=1).data
    p = p.cpu()
    top_ch = np.arange(len(model.chars))
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())
    return model.idx2char[char], h

In [16]:
def generate(nemodelt, prime='Мен', line_number=14):
    model.cuda()
    model.eval()
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h,)
    chars.append(char)
    n = 0; prev_ch = ""
    while not (n >= line_number and prev_ch in "?!.") and line_number < 100:
        char, h = predict(model, chars[-1], h)
        chars.append(char)
        if char == "\n":
            n += 1
        prev_ch = char
    return ''.join(chars)

In [17]:
print(generate(model, 'Мен', 18))

Мен кей кұңса, 
Ақсалығы балқылдап. 
Жаледен қығыр айтақ қарғанға.
Қай ақық қорғансыз шұй карбай.
Қай атында  аппылдап, 
Арасында құлын-тай 
Айнала шауып бұлтылдап, 
пандынты тұр балаңғы 
Қал-ыңыші қалым алтқан 
Күліші шағып жұры тіл, 
Қалыққан барған пел тарған 
Асыа толған оңық қарғайдап асынпан 
Бер жаңған талтым бүр іуде жасынан.
Ое үңіле көрі тоқса сотған қойға,
Қар-- апық, бүркіт – өапең тон тағынға.
Қайды тот - пашы үті  ылшы алға, 
Көренееі шіріп қалылып қойған дап 
Көріпер қарқаң босыса,
Колынқыз көлі бор жерген 
Етім тініп тоңқалдап.


In [18]:
print(generate(model, 'Дала', 18))

Далалын 
Салпы тат  қарыптіт есее қалман.
О да бол қасақ ақалда, моз барғанда. 
Аузын ашса, ыяқуқтап,
Қасысын қалуындап,
Бар баламын қылтылдап, 
Ат, айғырлар, биелер 
Бүйірші шығыы  бар жүлеп, 
Ақырығын балқын деп, 
Орынуыры қарасы қай, 
Жаныққандар, бәс еуеп, 
Астасын қанданға.
Бар бала ашы  ылынып, 
Үлден ит пей айтаққа. 
Колмып па едің жол тосып 
Жолығуғғы шалып бал жібек 
ал жобанды.
Қатда толған балтық шеп баршадан. 
Бар жешін ақтатай соғ алы сайғы.


In [19]:
print(generate(model, 'Мысал', 18))

Мысал қысан адім азға а ша ар, қызар бал.
Күйі тіп, тәрі шауда асық шынып.
Жасы үшін күлітде сөл ау санға.
Жасы аз патар етітсез қыр шан едес, 
Күйлесем, сөрдесем сен тармайда.
Қай марым бір кімір сел імдемін.
Бай масым бөрмерген сес белем, 
Жеп атпан катдан жүқ карша кесек.
Көзін аует көздемес. 
Үттен де ұядшап,
Қазлықын көлген де, қер шенем.
Жандаң ешы қай емын сан барға, 
Балуан да ам,аршы меп, алқалда. 
Көрімішер сара қал патып дыпға.
Мандан тоң ақ тізін, күйді тастап, 
Қалтықар кылып баз жерег, сысыл деп. 
Жендер өзыпқа қалы соң терпедім.
Ере төрікде келек сонда.
Қансақ күлген қошынған де - бірмерме,, 
Ықталма ас сап кем емім сез келім.


In [21]:
print(generate(model, 'Ой', 18))

Ой тіл аутағын.
Қайды алын қолынып, біл қалшанға. 
Бұрғаны шалқын бімер ек ша шағын, 
От ағына  олдан, көр ееі жұры 
Қетірде толсың бос анау, 
Қалрынғы дас- бар бер, 
Қолсысынға кисіп бал, 
Санжайлы сап болтындай. 
Байғаншы еш, шабыйдап, 
Өзепін қасық солдар - бал, ақ асаа.
Бүреді - бір ақылсан, Бұр ақанмай, 
Манақ бұлаң, босыны - өле сұрса, 
Таптағынтып болыны қылтылдан  
Са солықбан бәлі -өз,,
Қан ызылар  алығып.
Кұл ақақа қосып таң едеп, 
Үлгенен балдап бір аутеп, 
Мандынар балған бой шауасқа.
Бар - ақыл шаттан түркі жайыа еден.
